In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from keras.preprocessing.sequence import pad_sequences
import re
device_gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device_gpu)
from sklearn.model_selection import train_test_split 
import os,glob
import math
from torch.autograd import Variable
import librosa
import librosa.display
from librosa.filters import mel as librosa_mel_fn
import os,glob
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import copy

/home/jayesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jayesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jayesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jayesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [2]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model,inference, max_seq_len=3000):
        super(PositionalEncoder,self).__init__()
        #print(max_seq_len)
        self.inference=inference
        self.d_model = d_model
        self.alpha_factor=nn.Parameter(torch.diag(torch.ones(max_seq_len)))
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] =math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] =math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)
    def forward(self, x):
        #print(x.shape)
        #scaling
        self.pe=torch.matmul(self.alpha_factor,self.pe)
        #print(self.pe.shape, x.shape)
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #print(x.shape)
        #add constant to embedding

        seq_len = x.size(1)
        # print(x.shape)
        #print("ll",seq_len)
        # x=x.to(device_gpu)
        if self.inference == False:
          x = x + Variable(self.pe[:,:seq_len], requires_grad=False).cuda()
        else:
          x = x + Variable(self.pe[:,:seq_len], requires_grad=False).to('cpu')
        #print("hg",x.shape, self.pe[:,:seq_len].shape)
        return x

In [3]:
class Embedder(nn.Module):
    def __init__(self,  d_model, max_seq_len=200):
      super().__init__()
      self.embed = nn.Embedding(max_seq_len, d_model,padding_idx=0)
    def forward(self, x):
      return self.embed(x)

In [4]:
class Encoder_prenet(nn.Module):
  def __init__(self,d_model,max_seq_len):
    super(Encoder_prenet,self).__init__()
    convolutions=[]
    for _ in range(3):
      conv=nn.Sequential(
          nn.Conv1d(in_channels=d_model,out_channels=d_model,kernel_size=5,padding=2),
          nn.BatchNorm1d(d_model),
          nn.ReLU(),
          nn.Dropout(0.1)
      )
      convolutions.append(conv)
    self.convolutions=nn.ModuleList(convolutions)
    self.projection=nn.Linear(max_seq_len,max_seq_len)
  def forward(self,x):
    # print(x.shape)
    x=x.reshape(x.shape[0],x.shape[2],x.shape[1])
    for conv in self.convolutions:
      x=conv(x)
      
    # x = x.view(x.size(0), -1)
    
    x=self.projection(x)
    x=x.transpose(1,2)
    return x

In [5]:
class Multi_Head_Attention(nn.Module):
  def __init__ (self,heads,d_model,inference=False,dropout=0.1):
    super(Multi_Head_Attention,self).__init__()
    self.inference=inference
    self.heads=heads
    self.d_k=d_model//heads
    self.q_linear=nn.Linear(d_model,d_model)
    self.v_linear=nn.Linear(d_model,d_model)
    self.k_linear=nn.Linear(d_model,d_model)
    self.out = nn.Linear(d_model, d_model)
    self.dropout=nn.Dropout(dropout)
    self.d_model=d_model

  def forward(self,q,k,v,mask=None):
   
    bs_q=q.size(0)
    bs_v=v.size(0)
    bs_k=k.size(0)
    q=q.type(torch.FloatTensor)
    v=v.type(torch.FloatTensor)
    k=k.type(torch.FloatTensor)
    
    if self.inference == False:
      q=q.to(device_gpu)
      v=v.to(device_gpu)
      k=k.to(device_gpu)
    
    q=self.q_linear(q).view(bs_q,-1,self.heads,self.d_k)
    v=self.v_linear(v).view(bs_v,-1,self.heads,self.d_k)
    k=self.k_linear(k).view(bs_k,-1,self.heads,self.d_k)
    q=q.transpose(1,2)
    v=v.transpose(1,2)
    k=k.transpose(1,2)
    # calculate attention using function we will define next
    scores = attention(q=q, k=k, v=v, d_k=self.d_k, inference=self.inference,mask=mask, dropout=self.dropout)
    
    # concatenate heads and put through final linear layer
    concat = scores.transpose(1,2).contiguous().view(bs_q, -1, self.d_model)
    output = self.out(concat)
    
    return output

In [6]:
  def attention(q, k, v, d_k, inference=False,mask=None, dropout=None):
    #print(q.shape,k.transpose(-2,-1).shape)
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    
    if mask is not None:
      size=scores.size(2)
      #print(size)
      np_mask = np.triu(np.ones((1, size, size),dtype='uint8'),k=1)
      np_mask=torch.FloatTensor(np_mask)
      #print(np_mask)
      np_mask = np_mask.unsqueeze(1)
      if inference==False:
        np_mask=np_mask.to(device_gpu)
      scores = scores.masked_fill(np_mask == 1, -1e9)
    scores = F.softmax(scores, dim=-1,dtype=torch.float)
    if dropout is not None:
        scores = dropout(scores)
    output = torch.matmul(scores, v)
    return output
    

In [7]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super(Norm,self).__init__()
        self.size = d_model
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        # self.apha , self.bias , self.eps = self.alpha.to(device_gpu) , self.bias.to(device_gpu), self.eps.to(device_gpu) 
        # x=x.to('cpu')
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True))/(x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

In [8]:
class FFN(nn.Module):
  def __init__(self, d_model, d_ff=2048, dropout = 0.1):
    super(FFN,self).__init__() 
    # We set d_ff as a default to 2048
    self.linear_1 = nn.Linear(d_model, d_ff)
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model)
  def forward(self, x):
    x = self.dropout(F.relu(self.linear_1(x)))
    x = self.linear_2(x)
    return x

In [ ]:
class Encoder_layer(nn.Module):
  def __init__(self,d_model,heads,inference,dropout=0.1):
    super(Encoder_layer,self).__init__()
    self.norm_1=Norm(d_model)
    self.norm_2=Norm(d_model)
    self.ffn=FFN(d_model)
    self.attn=Multi_Head_Attention(heads,d_model,inference)
    self.dropout_1=nn.Dropout(dropout)
    self.dropout_2=nn.Dropout(dropout)
  def forward(self,x):
    # x=x.to('cpu')
    
    x2=self.norm_1(x)
   
    x=x+self.dropout_1(self.attn(x2,x2,x2))
    x2=self.norm_2(x)
    x=x+self.dropout_2(self.ffn(x2))
    return x

In [ ]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [ ]:
class Encoder(nn.Module):
  def __init__(self,d_model,heads,max_seq_len,inference,N):
    super(Encoder,self).__init__()
    self.N=N
    self.embed=Embedder(d_model)
    self.prenet=Encoder_prenet(d_model,max_seq_len)
    self.pe=PositionalEncoder(d_model,inference)
    self.layers=get_clones(Encoder_layer(d_model,heads,inference),N)
    self.norm=Norm(d_model)

  def forward(self,x):
    x=self.embed(x)
    x=self.prenet(x)
    x=self.pe(x)
    for i in range(self.N):
      x=self.layers[i](x)
    x=self.norm(x)
    return x

In [ ]:
class Decoder_prenet(nn.Module):
  def __init__(self,d_model,dropout=0.1):
    super(Decoder_prenet,self).__init__()
    self.linear_1=nn.Linear(80,256)
    self.linear_2=nn.Linear(256,256)
    self.linear_3=nn.Linear(256,d_model)
    self.dropout_1=nn.Dropout(dropout)
    self.dropout_2=nn.Dropout(dropout)
  def forward(self,x):
    

    x=self.dropout_1(F.relu(self.linear_1(x)))
    #print(x.shape)
    x=self.dropout_2(F.relu(self.linear_2(x)))
    x=self.linear_3(x)
    # x=x.unsqueeze(0)
    # x=x.transpose(1,2)
    
    return x

In [ ]:
class Decoder_Layer(nn.Module):
  def __init__(self,d_model,heads,inference,dropout=0.1):
    super(Decoder_Layer,self).__init__()
    self.norm_1=Norm(d_model)
    self.norm_2=Norm(d_model)
    self.norm_3=Norm(d_model)
    self.att_1=Multi_Head_Attention(heads,d_model,inference)
    self.att_2=Multi_Head_Attention(heads,d_model,inference)
    self.ffn=FFN(d_model)
    self.dropout_1=nn.Dropout(dropout)
    self.dropout_2=nn.Dropout(dropout)
    self.dropout_3=nn.Dropout(dropout)

  def forward(self,x,e_output,mask=1):
    # x=x.to('cpu')    
    x2=self.norm_1(x)
    
    
    x=x+self.dropout_1(self.att_1(x2,x2,x2,mask))
    x2=self.norm_1(x)
    
    x=x+self.dropout_2(self.att_2(x2,e_output,e_output))
    
    x2=self.norm_3(x)
    x=x+self.dropout_3(self.ffn(x2))
    return x

In [ ]:
class Decoder(nn.Module):
  def __init__(self,d_model,heads,inference,N):
    super(Decoder,self).__init__()
    self.N=N
    self.pe=PositionalEncoder(d_model,inference)
    self.prenet=Decoder_prenet(d_model)
    self.layers=get_clones(Decoder_Layer(d_model,heads,inference),N)
    self.norm=Norm(d_model)
  def forward(self,x,e_output,temp=1):
    x=self.prenet(x)

    x=self.pe(x)
    for i in range(self.N):
      x=self.layers[i](x,e_output)
    # print(x.shape)
    return self.norm(x)

In [ ]:
df_1=pd.read_pickle("/content/drive/MyDrive/Text-To-Speech_HINDI/chosen_files.pkl")
df_1["mel_output"].values[0].shape

torch.Size([80, 1639])

In [ ]:
class Linear_Projection(nn.Module):
  def __init__(self,d_model,channels):
    super(Linear_Projection,self).__init__()
    self.linear=nn.Linear(d_model,channels)
    torch.nn.init.xavier_normal_(self.linear.weight)
  def forward(self,x):
    # x=x.type(torch.LongTensor)
    return self.linear(x)

In [ ]:
class Conv_layer(nn.Module):
  def __init__(self,input,output,kernel,stride,pad,dilation,w_init_gain="linear"):
    super(Conv_layer,self).__init__()
    if pad is None:
      assert(kernel % 2 == 1)
      pad = int(dilation * (kernel - 1) / 2)
    self.cnn_layer=nn.Conv1d(in_channels=input,out_channels=output,kernel_size=kernel,stride=stride,padding=pad,dilation=dilation)
    torch.nn.init.xavier_uniform(self.cnn_layer.weight,gain=torch.nn.init.calculate_gain(w_init_gain))

  def forward(self,x):
    # x=x.type(torch.LongTensor)
    return self.cnn_layer(x)

In [ ]:
class CNN_projection(nn.Module):
  def __init__(self,channels,output,kernel,dropout=0.1):
    super(CNN_projection,self).__init__()
    self.dropout=nn.Dropout(dropout)
    self.cnn_model=nn.ModuleList()
    self.cnn_model.append(
        nn.Sequential(Conv_layer(channels,output,kernel,1,int((kernel-1)/2),1,"tanh"),
                      nn.BatchNorm1d(output))
    )
    for i in range(1,4):
      self.cnn_model.append(
          nn.Sequential(Conv_layer(output,output,kernel,1,int((kernel-1)/2),1,"tanh"),
                      nn.BatchNorm1d(output))
    )
    
    self.cnn_model.append(
        nn.Sequential(Conv_layer(output,channels,kernel,1,int((kernel-1)/2),1,"linear"),
                      nn.BatchNorm1d(channels))
    )

  def forward(self,x):
    for i in range(len(self.cnn_model)-1):
      x=self.dropout(torch.tanh(self.cnn_model[i](x)))
    x=self.dropout(self.cnn_model[-1](x))
    return x

In [ ]:
class Postnet(nn.Module):
  def __init__(self,d_model,channels_mel,channels_stop,kernel,output):
    super(Postnet,self).__init__()
    self.mel_linear=Linear_Projection(d_model,channels_mel)
    self.stop_linear=Linear_Projection(d_model,channels_stop)
    self.layers_cnn=CNN_projection(channels_mel,output,kernel)
    
  def forward(self,x):
    
    x1=self.stop_linear(x)
    x2=self.mel_linear(x)
    x2=x2.transpose(1,2)
    x3=self.layers_cnn(x2)
  
    x3=x2+x3
    return x1,x2,x3

  def inference(self,x):
    # print("as",x.shape)
    x1=self.stop_linear(x)
    x2=self.mel_linear(x)
    x2=x2.transpose(1,2)
 
    return x1,x2

In [ ]:
filter_length=1024
hop_length=256
win_length=1024
n_mel_channels=80
mel_fmin=0.0
mel_fmax=8000.0

class Model(nn.Module):
  def __init__(self,d_model,heads,max_seq_len,channels_mel,channels_stop,inference=False,kernel=5,output=512,N=6):
    super(Model, self).__init__()
    self.encoder=Encoder(d_model,heads,max_seq_len,inference,N=6)
    self.decoder=Decoder(d_model,heads,inference,N=6)
    self.postnet=Postnet(d_model,channels_mel,channels_stop,kernel,output=512)
    self.layers_cnn=CNN_projection(channels_mel,output,kernel)

  def forward(self,x,y):
    print(x.shape)
    y=y.float()
    y=y.transpose(1,2)
    y=torch.cat((torch.zeros(y.shape[0],1,80).to(device_gpu),y),dim=1)

    e_output=self.encoder(x)
    d_output=self.decoder(y,e_output)
    outputs=self.postnet(d_output)
    return outputs

  def inference(self,input):
    text_input=text_phonemes(MAX_LEN=10,text=input)[0]
    x=torch.LongTensor(text_input)
    x.squeeze(1)


    y=torch.zeros(1,1,80)
    e_output=self.encoder(x)
    a=[]
    for i in range(30):
      d_output=self.decoder(y,e_output)
      outputs=self.postnet.inference(d_output)
      # print(y.shape,outputs[1].transpose(1,2).shape)
      a.append(outputs[1])
      # print(i,torch.sigmoid(outputs[0].transpose(1,2))[0][0][0])
      if torch.sigmoid(outputs[0].transpose(1,2))[0][0][0]>0.6:
        print(i)
        break
      y=outputs[1].transpose(1,2)
    out=a[0]
    for i in range(1,len(a)):
      out=torch.cat((out,a[i]),dim=2)
    print(out.shape)
    outputs=self.layers_cnn(out)
    
    return outputs

In [ ]:
net=Model(32,8,10,80,1,inference=True,N=6)
x=torch.rand(1,10)
x=x.type(torch.LongTensor)
x=x.to(device_gpu)
i=torch.rand(1,80,16)
i=i[:,:,:15]
i=i.to(device_gpu)
# net=net.to(device_gpu)
# a=net(x,i)
net.eval()
a=net.inference('यतीन्द्र का सुर की बारादरी')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning:

nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.



torch.Size([1, 80, 30])


In [ ]:
class Loss(nn.Module):
  def __init__(self,value):
    super(Loss,self).__init__()
    self.value=value
  
  def forward(self,model_outputs,targets):
    stop_output,mel_output_final,mel_ouput = model_outputs
    _,true_stop,true_mel = targets
    true_stop=true_stop.to(device_gpu)
    true_mel=true_mel.to(device_gpu)
    true_stop.requires_grad=False
    true_mel.requires_grad=False

    mel_loss=nn.MSELoss()(mel_output,true_mel)+nn.MSELoss()(mel_output_final,true_mel)
    stop_loss=nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([self.value]))(stop_output,true_stop)
    loss=mel_loss+stop_loss
    return loss


In [ ]:
df=pd.read_pickle("/content/drive/MyDrive/Text-To-Speech_HINDI/text_selected.pkl")

In [ ]:
df_1["filenumber_male"]=df_1["filenumber_male"].astype(int)
df["text_id"]=df["text_id"].astype(int)

In [ ]:
df_1=df_1.sort_values(by=['filenumber_male'])
df=df.sort_values(by=['text_id'])

In [ ]:
padded_audio=torch.LongTensor(df_1.shape[0],80,2671)
output_lengths = torch.FloatTensor(2671)
padded_audio.zero_()
for i in range(df_1.shape[0]):
  padded_audio[i,:,:df_1["mel_output"].values[i].size(1)]=df_1["mel_output"].values[i]
  output_lengths[i] = df_1["mel_output"].values[i].size(1)

padded_text=torch.LongTensor(df.shape[0],1,152)
for i in range(df.shape[0]):
  padded_text[i,:,:]=torch.Tensor(df["text_processed"].values[i])
padded_text=padded_text.squeeze(1)

stop_vector=torch.LongTensor(df_1.shape[0],2671)
stop_vector.zero_()
for i in range(df_1.shape[0]):
  stop_vector[i,df_1["mel_output"].values[i].size(1)-1:]=1

In [ ]:
train_text,validation_text,train_audio,validation_audio=train_test_split(padded_text,padded_audio,random_state=2018, test_size=0.1)
train_stop,validation_stop,_,_=train_test_split(stop_vector,padded_audio,random_state=2018, test_size=0.1)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 

batch_size = 64
train_data = TensorDataset(train_text, train_stop, train_audio)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_text, validation_stop, validation_audio)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=validation_text.shape[0])


In [ ]:
loss_values=[]

In [ ]:
# Keep only a single checkpoint, the best over test accuracy.
def save_checkpoint(state, is_best, filename='/content/drive/MyDrive/Text-To-Speech_HINDI/checkpoint.pth.tar'):
    """Save checkpoint if a new best is achieved"""
    if is_best:
        print ("=> Saving a new best")
        torch.save(state, filename)  # save checkpoint
    else:
        print ("=> Training Loss did not improve")

In [ ]:
resume_weights = '/content/drive/MyDrive/Text-To-Speech_HINDI/checkpoint.pth.tar'

In [ ]:
# If exists a best model, load its weights!
if os.path.isfile(resume_weights):
    #print("=> loading checkpoint '{}' ...".format(resume_weights))
    if device_gpu:
        checkpoint = torch.load(resume_weights)
    else:
        # Load GPU model on CPU
        checkpoint = torch.load(resume_weights,
                                map_location=lambda storage,
                                loc: storage)
    start_epoch = checkpoint['epoch']
    best_accuracy = checkpoint['best_accuracy']
    net.load_state_dict(checkpoint['state_dict'])
    print("=> loaded checkpoint '{}' (trained for {} epochs)",checkpoint['epoch'],best_accuracy,start_epoch)

In [ ]:
import random
best_accuracy=torch.FloatTensor([0.0005])
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
net=Model(512,8,152,2671,80,1,N=6)
net=net.to(device_gpu)
for epoch_i in range(0, 2):
  for i, data in enumerate(train_dataloader):
    text=data[0].long().to(device_gpu)
    stop=data[1].long().to(device_gpu)
    audio=data[2].to(device_gpu)
    
    net.zero_grad() 
    outputs=net(text,audio)
    loss_model=Loss()
    loss=loss_model(outputs,data)
    total_loss += loss.item()/64 
    loss.backward(retain_graph=True)
    torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
  avg_train_loss = total_loss / len(train_dataloader)
  acc = torch.FloatTensor([avg_train_loss])
  # Get bool not ByteTensor
  is_best = bool(acc.numpy() < best_accuracy.numpy())
  # Get greater Tensor to keep track best acc
  best_accuracy = torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
  save_checkpoint({
                  'epoch': start_epoch + epoch_i + 1,
                  'state_dict': model.state_dict(),
                  'best_accuracy': best_accuracy
              }, is_best)
  loss_values.append(avg_train_loss)


In [ ]:
import re

def text_phonemes(MAX_LEN=153,text="यतीन्द्र मिश्र का सुर की बारादरी, इस नाम से, पेंग्विन यात्रा से शीघ्र प्रकाश्य पुस्तक से लिया गया है, जो उनकी कला, स्थानीय परम्परा, उनके व्यक्तित्व को, एक साथ पढ़ती हैं"):
  vowels=["ा","ि","ी","ु","ू","ृ","े","ै","ो","ौ","ँ",'ं','ॆ',"ː","आ","इ","ई","उ","ऊ","ऋ","ए","ऐ","ओ","औ","अः","अं"]
  full_consonants=["क","ख","ग","घ","ङ","च","छ","ज","झ","ञ","ट","ठ","ड","ढ","ण","त","थ","द","ध","न","प","फ","ब","भ","म","य","र","ल","ळ","व","श","ष","स","ह"]
  special_vowels=["ः","ं","ँ"]
  mapping={}
  mapping["अ"]="ə"
  mapping["आ"]="a:"
  mapping["इ"]="ɪ"
  mapping["ई"]="i:"
  mapping["उ"]="ʊ"
  mapping["ऊ"]="u:"
  mapping["ऋ"]="ɻ̩"
  mapping["ए"]="e:"
  mapping["ऐ"]="ɛ:"
  mapping["ओ"]="o:"
  mapping["औ"]="ɔ:"
  #mapping["अँ"]="/ə̃/"
  #mapping["अः"]="/əɦə/"
  mapping["अं"]="əm"
  #mapping["ऑ"]="/ɒ/"


  mapping["ा"]="a:"
  mapping["ि"]="ɪ"
  mapping["ी"]="i:"
  mapping["ु"]="ʊ"
  mapping["ू"]="u:"
  mapping["ृ"]="ɻ̩"
  mapping["े"]="e:"
  mapping["ै"]="ɛ:"
  mapping["ो"]="o:"
  mapping["ौ"]="ɔ:"
  #mapping["ँ"]="/ə̃/"
  #mapping["ː"]="/əɦə/"
  mapping['ं']="əm"
  mapping["ॆ"]="e:"

  mapping["क"+"्"]="k"
  mapping["ख"+"्"]="kʰ"
  mapping["ग"+"्"]="g"
  mapping["घ"+"्"]="gʰ"
  mapping["ङ"+"्"]="ŋ"
  mapping["च"+"्"]="tʃ"
  mapping["छ"+"्"]="tʃʰ"
  mapping["ज"+"्"]="dʒ"
  mapping["झ"+"्"]="dʒʰ"
  mapping["ञ"+"्"]="ɲ"
  mapping["ट"+"्"]="ʈ"
  mapping["ठ"+"्"]="ʈʰ"
  mapping["ड"+"्"]="ɖ"
  mapping["ढ"+"्"]="ɖʰ"
  mapping["ण"+"्"]="ɳ"
  mapping["त"+"्"]="t̪"
  mapping["थ"+"्"]="t̪ʰ"
  mapping["द"+"्"]="d̪"
  mapping["ध"+"्"]="d̪ʰ"
  mapping["न"+"्"]="n"
  mapping["प"+"्"]="p"
  mapping["फ"+"्"]="pʰ"
  mapping["ब"+"्"]="b"
  mapping["भ"+"्"]="bʰ"
  mapping["म"+"्"]="m"
  mapping["य"+"्"]="j"
  mapping["र"+"्"]="ɾ"
  mapping["ल"+"्"]="l"
  mapping["व"+"्"]="ʋ"
  mapping["स"+"्"]="s"
  mapping["श"+"्"]="ʃ"
  mapping["ष"+"्"]="ʂ"
  mapping["ह"+"्"]="ɦ"
  mapping["ळ"+"्"]="ɭ̆ɭ̆"
  mapping[" "]="SIL"


  #text="नमस्कार नमस्कार नमस्कार नमस्कार नमस्कार नमस्कार नमस्कार नमस्कार "
  line=text.split("\n")
  for i in range(len(line)):
    line[i]=line[i].strip()
  # print(line)
  b=[]

  for k in range(len(line)):
    x=str(b)+str("_")+str(k)
    x=[]
    a=line[k]
    a_1=re.split(" ",a)
    #print(a_1)
    
    for i in range(len(a_1)):
      for j in range(len(a_1[i])):
      
        #print(a_1[i])
        x.append(a_1[i][j])
      x.append("  ")
    b.append(x)

  for i in range(len(b)):
    for j in range(len(b[i])):
      if b[i][j]=="अ" and b[i][j+1] in special_vowels:
        b[i][j]="अ"+b[i][j+1]
        b[i][j+1]="."
    if "." in b[i]:
      b[i].remove(".")

  for i in range(len(b)):
    for j in range(len(b[i])):
      if b[i][j]=="्":
        #print(b[i-1],i)
        b[i][j-1]=b[i][j-1]+b[i][j]
        b[i][j]=""

  for i in range(len(b)):
    for j in range(3*len(b[i])):
      if j < len(b[i]):
        if b[i][j] in full_consonants and b[i][j+1] not in vowels:
          b[i][j]=b[i][j]+"्"
          b[i].insert(j+1,"अ")

    if b[i][len(b[i])-2] in full_consonants:
      b[i][len(b[i])-2]=b[i][len(b[i])-2]+"्"
      b[i].insert(len(b[i])-1,"अ")

  c=[]
  indices=[]
  for i in range(len(b)):
    y=str(c)+str("_")+str(i)
    y=[]
    for j in range(len(b[i])):
      if b[i][j]!="":
        y.append(b[i][j])
    c.append(y)


  for k in range(len(c)):
    for i in range(1,len(c[k])):
      if c[k][i] in vowels and c[k][i-1] not in vowels:
        c[k][i-1]=c[k][i-1]+"्"

  phones=[]

  for i in range(len(c)):
    z=str(phones)+str("_")+str(i)
    z=[]
    values=''
    for char in c[i]:  
      if char=='  ':
        z.append(values)
        values=''
        
      else:
        for key,value in mapping.items():
          if char == key:
            values=values+value+" "
      
      
    phones.append(z)

  phones_final=[]
  for i in range(len(phones)):
    z=str(phones_final)+str("_")+str(i)
    z=[]
    for j in range(len(phones[i])):
      if len(phones[i][j])==0:
        break
      else:
        z.append(phones[i][j])
    
    phones_final.append(z)
  for j in range(len(phones_final)):
    for i in range(1,5*len(phones_final[j]),2):
      if i<len(phones_final[j]):
        phones_final[j].insert(i,"SIL")
  # print(phones_final)
  # phonemes=[]
  # for i in range(len(phones_final[0])):
  #   for j in range(len(phones_final[0][i])):
  #     phonemes.append(phones_final[0][i][j].split(" "))
  phonemes=[]
  temp_pho=[]
  for j in range(len(phones_final)):
    a=[]
    for i in range(len(phones_final[j])):
      a.append(phones_final[j][i])
    temp_pho.append(a)
  # print(temp_pho)
  for j in range(len(temp_pho)):
    x=[]
    for i in range(len(temp_pho[j])):
      temp_pho[j][i]=temp_pho[j][i].strip()
      x.append(temp_pho[j][i].split(" "))
    phonemes.append(x)
  # print(phonemes)
  phonemes_final=[]
  for i in range(len(phonemes)):
    y=[]
    for j in range(len(phonemes[i])):
      for k in range(len(phonemes[i][j])):
        y.append(phonemes[i][j][k])
    phonemes_final.append(y)


  vocab=[]
  for key,values in mapping.items():
    vocab.append(values)
  word_to_ix = {word: i+1 for i, word in enumerate(vocab)}


  phonemes_num=[]
  for j in range(len(phonemes_final)):
    a=[]
    for i in range(len(phonemes_final[j])):
      # print([word_to_ix[w] for w in phonemes_final[i]])
      a.append(word_to_ix[phonemes_final[j][i]])
    phonemes_num.append(a)

  # MAX_LEN = 100
  #print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
  #print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
  input_ids = pad_sequences(phonemes_num, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
  return input_ids,len(phonemes_num[0])